In [1]:
from TrainVoxelmorph import train_vxm_model
from DataHandler import DataHandler
import voxelmorph as vxm
import SimpleITK as sitk
import numpy as np
import vxmlosses
import tensorflow as tf
import pytorchlosses
import torch
import devLossOld

batch_size = 8
batch_size_val = 12
epochs = 200
steps = 100
learning_rate = 0.001
nb_features = [[16, 32, 32, 32], [32, 32, 32, 32, 32, 16, 16]]
#nb_features = [[16,16,32,32],[32,32,16,16]]
resampling = False
dataset = 'synthetic'
multi_gpu = True
dh = DataHandler()
if dataset == 'synthetic':
    dh.get_synthetic_data(
        fixed_path=
        '/home/cschellenberger/datam2olie/synthetic/orig/t3/Synthetic_CT/',
        moving_path=
        '/home/cschellenberger/datam2olie/synthetic/orig/t1/Synthetic_MR/')
elif dataset == 'L2R':
    dh.get_synthetic_data(
        fixed_path=
        '/home/cschellenberger/Documents/L2R_Resampled/L2R_Task1_CT/',
        moving_path=
        '/home/cschellenberger/Documents/L2R_Resampled/L2R_Task1_MR/')
elif dataset == 'mnist':
    dh.get_mnist_data(select_number=5)
else:
    raise NotImplementedError(
        f'{dataset} is not implemented yet please select one of the following losses [mnist, synthetic]'
    )                                       

nmiDev = vxm.losses.MutualInformation()
nmiDevOld = devLossOld.MutualInformation()
nmi = vxmlosses.NMI(bin_centers = np.linspace(0, 1, num=16), vol_size = (256, 256, 128))
#nmiTorch = pytorchlosses.MutualInformation()

tf.Tensor(449.99994, shape=(), dtype=float32)
tf.Tensor(449.99994, shape=(), dtype=float32)


2022-03-01 06:45:51.826061: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-01 06:45:55.018227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 112 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:1d:00.0, compute capability: 7.5
2022-03-01 06:45:55.019217: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 112 MB memory:  -> device: 1, name: Quadro RTX 8000, pci bus id: 0000:1e:00.0, compute capability: 7.5
2022-03-01 06:45:55.020027: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2

In [2]:
## 
import tensorflow.keras.backend as K
moving_image_paths = dh.x_val
fixed_image_paths = dh.y_val
moving_image = sitk.ReadImage(moving_image_paths[0])
fixed_image = sitk.ReadImage(fixed_image_paths[0])
moving_image = sitk.GetArrayFromImage(moving_image)
fixed_image = sitk.GetArrayFromImage(fixed_image)
moving_image = moving_image.reshape((1, 256, 256, 128))
fixed_image = fixed_image.reshape((1, 256, 256, 128))

In [2]:
nmiLoss = nmi.loss(moving_image, moving_image)
print(K.sum(nmiLoss))
nmiLoss = nmi.loss(moving_image, fixed_image)
print(K.sum(nmiLoss))

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(-0.9707546, shape=(), dtype=float32)
tf.Tensor(-0.40738332, shape=(), dtype=float32)


In [5]:
## 
with tf.device('/cpu:0'):
    nmiDevLoss = nmiDev.loss(moving_image, moving_image)
    nmiDevLoss = tf.reduce_mean(nmiDevLoss, 1, keepdims=True)
    print(K.sum(nmiDevLoss))
    nmiDevLoss = nmiDev.loss(moving_image, fixed_image)
    nmiDevLoss = tf.reduce_mean(nmiDevLoss, 1, keepdims=True)
    print(K.sum(nmiDevLoss))

tf.Tensor(-0.9869835, shape=(), dtype=float32)
tf.Tensor(-0.44396466, shape=(), dtype=float32)


In [5]:
moving_image_torch = torch.from_numpy(moving_image)
fixed_image_torch = torch.from_numpy(fixed_image)
nmiTorchLoss = nmiTorch.forward(moving_image_torch, moving_image_torch)
print(nmiTorchLoss)
nmiTorchLoss = nmiTorch.forward(moving_image_torch, fixed_image_torch)
print(nmiTorchLoss)

tensor(-0.9721)
tensor(-0.4068)


In [4]:
nmiDevLossOld = nmiDevOld.loss(moving_image, moving_image)
print(K.sum(nmiDevLossOld))
nmiDevLoss = nmiDevOld.loss(moving_image, fixed_image)
print(K.sum(nmiDevLossOld))

tf.Tensor(-129.84944, shape=(), dtype=float32)
tf.Tensor(-129.84944, shape=(), dtype=float32)
